# Scrape the Reviews from Apartments.com

## Imports

In [1]:
import pandas as pd
from pathlib import Path

import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [101]:
import warnings
warnings.filterwarnings('ignore')

## Load DataFrame of Apartments

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
base_path = Path('drive', 'MyDrive', 'ApartmentRecommender')
data_path = base_path / 'data'
info_path = data_path / 'info.csv'

In [4]:
info_df = pd.read_csv(info_path)
info_df.drop('Unnamed: 0', axis=1, inplace=True)
info_df.head()

,names,urls,addresses
0,IMT Cherry Creek,https://www.apartments.com/imt-cherry-creek-de...,"755 S Dexter St, Denver, CO 80246"
1,Cirrus,https://www.apartments.com/cirrus-denver-co/9n...,"1590 Grove St, Denver, CO 80204"
2,Outlook DTC,https://www.apartments.com/outlook-dtc-denver-...,"5031 S Ulster St, Denver, CO 80237"
3,Sentral Union Station,https://www.apartments.com/sentral-union-stati...,"1770 Chestnut Pl, Denver, CO 80202"
4,Deco Apartments,https://www.apartments.com/deco-apartments-den...,"2180 S Colorado Blvd, Denver, CO 80222"


In [5]:
urls = info_df['urls']

## Scrape reviews

In [104]:
def get_reviews(url):
  headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
  r = requests.get(url, headers=headers)

  content = r.content
  soup = BeautifulSoup(content)

  output = {}

  total_score = soup.find('div', attrs={'class':'averageRating'})
  if total_score != None:
    total_score = total_score.text
  output['total_score'] = total_score

  for i, f in enumerate(soup.findAll('div', attrs={'class':'reviewContainerWrapper'})):
    output[i] = {}

    output[i]['stars'] = len(f.find('span').findAll('i', attrs={'class':'storyicon starFullStoryIcon'}))
    output[i]['date'] = f.find('span', attrs={'class':'reviewDateContainer'}).text 
    output[i]['title'] = f.find('h3', attrs={'class':'reviewTitle'}).text 
    output[i]['review'] = f.find('p', attrs={'class':'reviewText reviewFullText'}).text.strip()
    output[i]['response'] = f.find('p', attrs={'class':'comment'})
    if output[i]['response'] != None:
      output[i]['response'] = output[i]['response'].text.strip()

  return output

In [107]:
def convert_to_df(review_dict):
  output = pd.DataFrame(columns=['date', 'stars', 'title', 'review', 'response'])

  for k in review_dict.keys():
    if k != 'total_score':
      temp = pd.DataFrame(review_dict[k], index=[k])
      output = pd.concat([output, temp])

  return output

In [108]:
for i in info_df.index:
  print(i)
  url = info_df.loc[i, 'urls']
  reviews = get_reviews(url)
  review_df = convert_to_df(reviews)

  table_name = info_df.loc[i, 'names'] + '/reviews'
  table_name = table_name.replace(' ', '_')

  with pd.HDFStore('apartmentReviews.h5') as store:
    store.put(table_name, review_df)
print('Done!')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [109]:
!cp apartmentReviews.h5 drive/MyDrive/ApartmentRecommender/data